In [16]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint



#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [17]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_new = application_df.drop(columns=['EIN', 'NAME'])

In [18]:
# Determine the number of unique values in each column.
application_df_new.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [19]:
# Look at APPLICATION_TYPE value counts for binning
application_df_new['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [20]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = []
app_filter = application_df_new['APPLICATION_TYPE'].value_counts()[application_df_new['APPLICATION_TYPE'].value_counts() < 528]
application_types_to_replace.append(app_filter.index)  

# Replace in dataframe
for app in application_types_to_replace:
    application_df_new['APPLICATION_TYPE'] = application_df_new['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_new['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [21]:
# Look at CLASSIFICATION value counts for binning
counts = application_df_new['CLASSIFICATION'].value_counts()
counts 

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [22]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
cls_filter = application_df_new['CLASSIFICATION'].value_counts()[application_df_new['CLASSIFICATION'].value_counts() < 1883]
classifications_to_replace.append(cls_filter.index.tolist())  

# Replace in dataframe
for cls in classifications_to_replace:
    application_df_new['CLASSIFICATION'] = application_df_new['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_new['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [23]:
# Convert categorical data to numeric with `pd.get_dummies`
X = application_df_new.drop(columns='IS_SUCCESSFUL')
X = pd.get_dummies(X)

In [24]:
# Split our preprocessed data into our features and target arrays
y = application_df_new['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='sigmoid', input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='sigmoid'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation='sigmoid'))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=17, activation='sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 100)               4400      
                                                                 
 dense_6 (Dense)             (None, 50)                5050      
                                                                 
 dense_7 (Dense)             (None, 25)                1275      
                                                                 
 dense_8 (Dense)             (None, 17)                442       
                                                                 
 dense_9 (Dense)             (None, 1)                 18        
                                                                 
Total params: 11,185
Trainable params: 11,185
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Train the model
checkpoint_path = 'checkpoints_optimized/weights.{epoch:02d}.hdf5'
callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1, 
                                              period=5,
                                              )
fit_model = nn.fit(X_train_scaled, y_train, epochs=60, callbacks=[callback])

Epoch 1/60
804/804 [==============================] - 8s 6ms/step - loss: 0.6036 - accuracy: 0.6923
Epoch 2/60
804/804 [==============================] - 6s 7ms/step - loss: 0.5758 - accuracy: 0.7301
Epoch 3/60
804/804 [==============================] - 4s 5ms/step - loss: 0.5715 - accuracy: 0.7292
Epoch 4/60
804/804 [==============================] - 4s 4ms/step - loss: 0.5628 - accuracy: 0.7279
Epoch 5/60
801/804 [============================>.] - ETA: 0s - loss: 0.5596 - accuracy: 0.7308
Epoch 5: saving model to checkpoints_optimized/weights.05.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.5595 - accuracy: 0.7309
Epoch 6/60
804/804 [==============================] - 2s 3ms/step - loss: 0.5583 - accuracy: 0.7287
Epoch 7/60
804/804 [==============================] - 2s 2ms/step - loss: 0.5565 - accuracy: 0.7310
Epoch 8/60
804/804 [==============================] - 2s 2ms/step - loss: 0.5553 - accuracy: 0.7299
Epoch 9/60
804/804 [==============================] 

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5552 - accuracy: 0.7305 - 528ms/epoch - 2ms/step
Loss: 0.5552477240562439, Accuracy: 0.7304956316947937


In [30]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')